ORF Finding Method

The ORF finder is primarily composed of three functions

-   codons(seq, start_c)
-   ORF(condonPositions, min_len)
-   ORFfinder(dta, nucleotides, sel_gen, rev, min_len)

codons() will find all positions of start and stop codons and place them into three lists, one for each frame. Once found, the ORF() function will accept these lists and validate/pair the start and stop codons based on a set minimum length as well as where the previous ORF was found. For each valid start codon, multiple valid stop codons are paired in order to obtain 3 ORFs. Both of these functions are called repeatedly within the ORFfinder() function. Within this function, the ORFcheck() function also runs to determine how many genes the ORF finder finds and how many extraneous ORFs it finds.

<font color='blue'>Cell 1 Loading necessary packages

In [ ]:
## Upload data from Module 6 data folder: 

# all_gene_positions.txt
# all_incorrect_ORFS.csv
# full_sequence.fasta
# genome1.fasta
# genome1_gene_positions.txt
# genome2.fasta
# rev_comp.fasta

In [ ]:
import numpy as np
import pandas as pd

<font color='blue'>Cell 2 
Reading data

In [ ]:
#Read full forward direction genome
with open('full_sequence.fasta', 'r') as file:
    full_seq = file.read().replace('\n', '')

#The first 71 characters contain the genome’s #information (reference number etc.). This is removed.
full_seq = full_seq[71:]

#Read full reverse complement direction genome
with open('rev_comp.fasta', 'r') as file:
    full_rev = file.read().replace('\n', '')

#Remove the first line
full_rev = full_rev[82:]



# ORF finder

<font color='blue'>Cell 3
Finding codon positions

codons(): This function will find the positions at which a start codon or stop codon appears. This function will return two lists (starts and stops), each with three sublists that contain the found positions in three frames.

Inputs: 
-   seq: The sequence being considered, ie. forward or reverse complement sequences (full_seq and full_rev respectively)
-   start_c: The start codon considered. This function is called from another function that iterates through ATG, GTG, TTG

Inside the function:
-   stops: The stop codons that will be found
-   lst1: A list that will contain the start codon positions
-   lst2: A list that will contain the stop codon positions
      - Both lst1 and lst2 are a list of 3 lists where each sublist is the list of positions for each frame.
-   while(start < 3): For 'start' iterator values 0, 1, and 2, look at all codons (in frame) from beginning to the end of the sequence. If the codon is a match for the start codon of interest, add it to the corresponding lst1 list. If the codon is a match for any of the stop codons, add it to the corresponding list in lst2. Once the end of the sequence is reached, increment the 'start' iterator to look for codons in the next frame.

Outputs:
Returns lst1 and lst2 

In [ ]:
#ORF Finder part 1: Find codon positions
def codons(seq,start_c):
  stops = ["TAA","TGA","TAG"]   
  lst1 = [[],[],[]] #List for the start codons
  lst2 = [[],[],[]]  #List for the stop codons
  start = 0 #Counter for 3 optional frames
  
  while (start < 3): 
      for i in range(start,len(seq),3):
          #Codon extraction
          codon = seq[i:i+3] #Move through the seuqence 3 bases at a time
          if (codon == start_c): #If codon is a start codon, add the position to the list
                lst1[start].append(i+1)
          if (codon in stops): #If codon is a start codon, add the position to the list 
                lst2[start].append(i+1)
      start += 1  
  return lst1,lst2

<font color='blue'>Cell 4
Determining valid ORFs

ORF(): Pairs the start and stop codon positions found with the codons function to form valid ORF start-stop pairs.

Inputs:
*   min_len: The minimum length used for determining valid ORFs
*   lst1, lst2 are the outputs of the codons() function

Inside the function:
*   ORFstarts and ORFstops: Lists that will contain the starts and stops of the determined valid ORFs
*   all_frames: When looking for ORFs, all start codon positions for all frames will be put into one list. This allows to scan for valid ORF starts sequentially across all frames.
*   start_min: Starting from zero, this variable is updated to dictate the position from which the next ORF start will be considered. After a start codon has been paired with its stop codons, this variable is updated with the min_len and the look_back.
*   stop_count: The stop position for loop iterates until each valid start codon is paired with 3 stops so long as the min_len condition is met. 
*   look_back: This parameter is subtracted from the start_min variable to account for the possibility of overlapping ORFs

Outputs: 
-   ORFstarts and ORFstops: These two lists contain the start and stop positions of codons that form true ORFs. Each element in the ORFstarts list is paired with the corresponding element of the ORFstops list, eg. ORFstarts[1] and ORFstops[1] are the indices of the first ORF that was found by the ORF() function.


In [ ]:
#ORF Finder part 2: Determine valid ORFs
def ORF(lst1, lst2, min_len):
  ORFstarts = [] 
  ORFstops = []   
  #min_len = 40 
  start_min = 0     
  look_back = 5    
  max_fr_len = max([len(i) for i in lst1])

  #Put all start codons for all frames into one list and sort
  all_frames = lst1[0] + lst1[1] + lst1[2]
  all_frames.sort()
  
  for s_cod in all_frames:
    if s_cod > start_min:
      for s_frame in range(len(lst1)): 
        if s_cod in lst1[s_frame]:
          frame = s_frame  
          break
      stop_count = 0 

      for stp_cod in lst2[frame]:
        if stp_cod > s_cod + min_len:
          ORFstarts.append(s_cod)
          ORFstops.append(stp_cod)
          stop_count +=1 
        if stop_count > 2:
          start_min = start_min + min_len - look_back
          break

  return ORFstarts, ORFstops

<font color='blue'>Cell 5
ORF checking function

ORFcheck(): Checks which of the ORF start-stop pairs are coding (ie. found in the gene position data) and which are non coding (ie. not found in the gene position data).

Inputs: 
-   ORFstarts and ORFstops are lists from the ORF() function
-   gene_pos: The gene position data (either gene_position or gene_position_r)
-   rev: A True/False parameter that is set to True if looking at the reverse complement (with gene_position_r), and False otherwise

Inside the function:
-   wrong_flag: A flag variable that equals 1 for non coding ORFS and 0 for coding ORFs (used to control flow of the loops)
-   OC: start-stop pair being tested
*   corr: The number of found ORFs that are contained in the gene position data (used for diagnostic purposes)
*   corORF: A dataframe of the ORFs (start and end positions) that are contained in the gene position data
*   incorORF: A dataframe of the ORFs (start and end positions) that are not contained in the gene position data (non coding ORFs)

Outputs: Returns corr, corORF and incorORF

In [ ]:
#ORF finder part 3: Determining which found ORFs are coding and non coding
#Import gene position data 
all_gene_data = pd.read_csv('all_gene_positions.txt', header=None).to_numpy()    
gene_position = all_gene_data[0:2269,:]     #Data for forward direction
gene_position_r = all_gene_data[2269:,:]    #Data for reverse complement

def ORFcheck(ORFstr, ORFstp, gene_pos, rev):
  corr = 0       
  corORF = np.empty((0,2))   
  incorORF = np.empty((0,2)) 
  wrong_flag = 1    

  #Iterate through all codons in the ORFstart list
  for i in range(len(ORFstr)):
    wrong_flag = 1
    #OC is current ORF start stop pair that is being tested
    #If rev, subtract one from the start position (different indexing)
    if rev:
      OC = np.array([ORFstr[i]-1, ORFstp[i]+2])
    else:
      OC = np.array([ORFstr[i], ORFstp[i]+2])
    
    #The following 2 conditions check to see if OC is contained in the gene position data
    if len(np.where(gene_pos==OC)[0]) == 2:
      if np.where(gene_pos==OC)[0][0] == np.where(gene_pos==OC)[0][1]:
        #If the two conditions are met, then increment 'corr' and enter the ORF start and stop into the corORF array
        corr += 1
        OC = OC.reshape(1,-1)
        corORF = np.concatenate((corORF,OC), axis=0)
        #Set the wrong flag to 0 to avoid passing the next condition
        wrong_flag = 0
    
    #If the two conditions are not true, ie. wrong_flag is still 1, enter the ORF pair into the incorORF array
    if wrong_flag == 1:
      OC = OC.reshape(1,-1)
      incorORF = np.concatenate((incorORF,OC), axis=0)
  return corr, corORF, incorORF

<font color='blue'>Cell 6
Running the previous functions

ORFfinder(): This function simply runs the functions defined above and returns the dataframes of all coding and non coding ORFs found

Inputs:
*   dta: Either full_seq or full_rev (forward genome sequence or reverse complement genome sequence)
*   nucleotides: A parameter to limit the ORFfinder search to reduce processing time (ie. nucleotides = 10000 will only look at the first 10000 nucleotides in the genome)
*   sel_gen: The gene position data that is also limited by the 'nucleotides' parameter
*  rev and min_len as defined in previous functions

Outputs: Returns two arrays, one that holds the start-stop pairs for coding ORFs (all_corORFs) and one that holds the start-stop pairs for non coding ORFs (all_incorORFs)


In [ ]:
#ORF Finder part 4: Putting all functions together
def ORFfinder(dta, nucleotides, sel_gen, rev, min_len):
  #Initializing arrays that will contain the correctly and incorrectly
  #found ORFs
  all_corORFs = np.empty((0,2))
  all_incorORFs = np.empty((0,2))
  start_codons = ["ATG","GTG","TTG"]
  #Iterate the ORF finder functions over the start codons
  for start_cod in range(len(start_codons)):
    #Finding the locations of the start and stop codons
    lst1, lst2 = codons(dta[0:nucleotides], start_codons[start_cod])
    #Pair valid ORF start and stops locations
    ORFstarts, ORFstops = ORF(lst1,lst2, min_len)
    corr, corORF, incorORF = ORFcheck(ORFstarts, ORFstops, sel_gen, rev)
    all_corORFs = np.concatenate((all_corORFs,corORF),axis=0)
    all_incorORFs = np.concatenate((all_incorORFs,incorORF),axis=0)
  return all_corORFs, all_incorORFs

# ORF Finder with an E. coli genome



<font color='blue'>Cell 7
Finding ORFs on a section of the E. coli genome

genome_section(): This function takes inputs from the user and runs the ORFfinder() function. This function prints how many coding ORFs and non coding ORFs were found.

Output: Returns 2 arrays, one that contains all start-stop pairs for coding and non coding ORFs one the forward genome (all_f_pred), and one that contains all start-stop pairs for coding and non coding ORFs one the reverse complement genome (all_r_pred).

In [ ]:
def genome_section():
  #Get the necessary parameters from the user
  nucleotides = input("Please enter the number of nucleotides to analyze: ")
  min_len = input("Please enter the minimum length of ORFs to consider: ")
  min_len = int(min_len)
  nucleotides = int(nucleotides)
  #Limit the forward and reverse complement using the 'nucleotides' parameter
  forward_section = full_seq[0:nucleotides]
  reverse_section = full_rev[0:nucleotides]

  #Limiting the gene position data by the 'nucleotides' parameter 
  selected_genes_f = np.array([gene_position[i] for i in range(len(gene_position)) if gene_position[i,1] < nucleotides])
  selected_genes_r = np.array([gene_position_r[i] for i in range(len(gene_position_r)) if gene_position_r[i,1] < nucleotides])
  print("-----------------------------------------------------------\n")
  print("There are", str(len(selected_genes_f)), "genes on the forward strand")
  print("There are", str(len(selected_genes_r)), "genes on the reverse strand")
  print("-----------------------------------------------------------\n")

  #Running the ORFfinder function on the forward and reverse complement with the entered parameters
  f_corr, f_incorr = ORFfinder(full_seq, nucleotides, selected_genes_f, rev=False, min_len=min_len)
  r_corr, r_incorr = ORFfinder(full_rev, nucleotides, selected_genes_r, rev=True, min_len=min_len)

  #Printing the number of coding ORFs and non coding ORFs found
  print("Forward:")
  print("\t",str(len(f_corr)), "found correctly")
  print("\t",str(len(f_incorr)), "additional ORFs found")

  print("Reverse:")
  print("\t",str(len(r_corr)), "found correctly")
  print("\t",str(len(r_incorr)), "additional ORFs found")

  all_f_pred = np.concatenate((f_corr,f_incorr)).astype(int)
  all_r_pred = np.concatenate((r_corr,r_incorr)).astype(int)

  return all_f_pred, all_r_pred


all_f_pred, all_r_pred = genome_section()

Please enter the number of nucleotides to analyze: 500
Please enter the minimum length of ORFs to consider: 40
-----------------------------------------------------------

There are 0 genes on the forward strand
There are 0 genes on the reverse strand
-----------------------------------------------------------

Forward:
	 0 found correctly
	 24 additional ORFs found
Reverse:
	 0 found correctly
	 55 additional ORFs found


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


<font color='blue'>Cell 8
Running the previous functions on a fraction of the E. coli genome

This code block outputs the number of ORFs found along with tables of the first 5 ORFs for both the forward and the reverse complement genome sequences. It also contains the function test_genome_ORFs (), which obtains a fraction of the imported genome sequence for ORF testing.

In [ ]:
genome_filename = 'genome1.fasta'
def test_genome_ORFs(lim):
  test_genome = pd.read_csv(genome_filename,header=None).to_numpy()[0][0]
  test_genome = test_genome[0:int(len(test_genome)*lim)]

  #Finding reverse complement  
  test_genome_r = test_genome[::-1]
  tst_gen_rcomp = ''
  for i in test_genome_r:
    if i == 'A':
      tst_gen_rcomp += 'T'
    elif i == 'T':
      tst_gen_rcomp += 'A'
    elif i == 'C':
      tst_gen_rcomp += 'G'
    else:
      tst_gen_rcomp += 'C'
  test_genome_r = tst_gen_rcomp

  min_len = input("Please enter the minimum length of ORFs to consider: ")
  print('\n')
  min_len = int(min_len)

  start_codons = ["ATG","GTG","TTG"]
  for start_cod in range(len(start_codons)):
    lst1, lst2 = codons(test_genome, start_codons[start_cod])
    ORFstarts, ORFstops = ORF(lst1,lst2,min_len)
    forward_ORFs = [[ORFstarts[i],ORFstops[i]] for i in range(len(ORFstarts))]

    lst1_r, lst2_r = codons(test_genome_r, start_codons[start_cod])
    ORFstarts_r, ORFstops_r = ORF(lst1_r,lst2_r,min_len)

    reverse_ORFs = [[ORFstarts_r[i],ORFstops_r[i]] for i in range(len(ORFstarts_r))]

    forward_ORFs = pd.DataFrame(forward_ORFs)
    forward_ORFs = forward_ORFs.rename(columns={0: "ORF start location", 1: "ORF stop location"})
    reverse_ORFs = pd.DataFrame(reverse_ORFs)
    reverse_ORFs = reverse_ORFs.rename(columns={0: "ORF start location", 1: "ORF stop location"})

  print("-----------------------------------------------------------\n")
  print("ORFs found on the forward direction:            ", len(ORFstarts))
  print("ORFs found on the reverse complement direction: ", len(ORFstarts_r))

  return forward_ORFs, reverse_ORFs


size_limit = float(input("Enter a value between 0 and 1 to limit the genome"))
forward_ORFs, reverse_ORFs = test_genome_ORFs(size_limit)

print("-----------------------------------------------------------")

print("\nThe first 5 ORFs found on the forward direction: \n")
print(forward_ORFs.iloc[0:5,:])


print("-----------------------------------------------------------")

print("\nThe first 5 ORFs found on the reverse complement direction: \n")
print(reverse_ORFs.iloc[0:5,:])


Enter a value between 0 and 1 to limit the genome.05
Please enter the minimum length of ORFs to consider: 40


-----------------------------------------------------------

ORFs found on the forward direction:             3577
ORFs found on the reverse complement direction:  3254
-----------------------------------------------------------

The first 5 ORFs found on the forward direction: 

   ORF start location  ORF stop location
0                  10                337
1                  10                346
2                  10                364
3                  65                122
4                  65                263
-----------------------------------------------------------

The first 5 ORFs found on the reverse complement direction: 

   ORF start location  ORF stop location
0                   6                525
1                   6                540
2                   6                612
3                  88                229
4                  88             